In [ ]:
! pip install --upgrade tensorflow

In [ ]:
! pip install --upgrade tensorflow-hub

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split

In [2]:
path='data.csv'
df=pd.read_csv(path)
df.head()

,URLs,Headline,Body,Label
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",1


In [3]:
df_new=df.fillna(" ")

In [4]:
y=df_new['Label'].astype(float) 
X=df_new['Headline'] 

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [6]:
# embedding = "https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1"
embedding = 'https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1'
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(X_train)

<tf.Tensor: shape=(3006, 20), dtype=float32, numpy=
array([[-0.73205507, -0.6554056 ,  1.1833174 , ..., -0.24447455,
        -1.1456059 ,  0.06286386],
       [-0.280999  , -0.81283945, -0.22278275, ..., -1.1509861 ,
        -0.6287009 ,  0.6653349 ],
       [-0.32817727,  0.2185772 , -0.49883842, ..., -0.14024827,
         0.2730054 ,  0.12530677],
       ...,
       [-1.8811165 ,  0.06307185,  1.2109767 , ...,  1.2666894 ,
         0.5034226 , -0.15597394],
       [-1.8830885 , -0.16736971, -0.1791039 , ...,  1.8812814 ,
         1.3526081 ,  1.1140953 ],
       [-0.04982694,  0.17166089,  0.6246027 , ..., -0.10046394,
        -1.2477318 ,  0.83508855]], dtype=float32)>

In [7]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(60, activation='relu'))
model.add(tf.keras.layers.Dense(25, activation='relu'))
model.add(tf.keras.layers.Dense(1))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 60)                1260      
_________________________________________________________________
dense_1 (Dense)              (None, 25)                1525      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 26        
Total params: 402,831
Trainable params: 402,831
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [10]:
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test,y_test))

Epoch 1/10
94/94 [==============================] - 0s 3ms/step - loss: 0.1640 - accuracy: 0.9361 - val_loss: 0.2817 - val_accuracy: 0.9023
Epoch 2/10
94/94 [==============================] - 0s 2ms/step - loss: 0.1268 - accuracy: 0.9508 - val_loss: 0.2889 - val_accuracy: 0.9023
Epoch 3/10
94/94 [==============================] - 0s 2ms/step - loss: 0.0962 - accuracy: 0.9634 - val_loss: 0.2919 - val_accuracy: 0.9043
Epoch 4/10
94/94 [==============================] - 0s 2ms/step - loss: 0.0725 - accuracy: 0.9724 - val_loss: 0.3079 - val_accuracy: 0.9043
Epoch 5/10
94/94 [==============================] - 0s 2ms/step - loss: 0.0497 - accuracy: 0.9817 - val_loss: 0.3134 - val_accuracy: 0.8983
Epoch 6/10
94/94 [==============================] - 0s 2ms/step - loss: 0.0349 - accuracy: 0.9874 - val_loss: 0.3411 - val_accuracy: 0.9013
Epoch 7/10
94/94 [==============================] - 0s 2ms/step - loss: 0.0242 - accuracy: 0.9947 - val_loss: 0.3697 - val_accuracy: 0.9053
Epoch 8/10
94/94 [==

In [11]:
print(model.evaluate(X_test,y_test))

32/32 [==============================] - 0s 1ms/step - loss: 0.4561 - accuracy: 0.9063
[0.456069678068161, 0.9062811732292175]


In [12]:
model.save("savemodel")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: savemodel/assets


INFO:tensorflow:Assets written to: savemodel/assets


In [13]:
from sklearn.metrics import classification_report
prediction = model.predict(X_test)
prediction = np.array([1 if row>=0.5 else 0 for row in prediction])
print(classification_report(y_test, prediction))

              precision    recall  f1-score   support

         0.0       0.89      0.94      0.91       535
         1.0       0.93      0.87      0.90       468

    accuracy                           0.91      1003
   macro avg       0.91      0.90      0.91      1003
weighted avg       0.91      0.91      0.91      1003



In [ ]:
# import mlflow
# import mlflow.tensorflow

# mlflow.tensorflow.autolog()

In [ ]:
# loaded_model = tf.keras.models.load_model('savemodel')

In [ ]:
loaded_model.summary()

In [18]:
# mlflow.keras.save_model(model, "keras_model")

In [ ]:
# ! pip install keras

In [ ]:
# loaded_keras_model = mlflow.keras.load_model("keras_model", custom_objects={'KerasLayer':hub.KerasLayer})

In [ ]:
# X_test[:2].to_json(orient="split")

In [15]:
tf.compat.v1.keras.experimental.export_saved_model(model, 'model.h5')

Instructions for updating:
Please use `model.save(..., save_format="tf")` or `tf.keras.models.save_model(..., save_format="tf")`.


Instructions for updating:
Please use `model.save(..., save_format="tf")` or `tf.keras.models.save_model(..., save_format="tf")`.


Instructions for updating:
Simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


Instructions for updating:
Simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


...

INFO:tensorflow:No assets to save.


INFO:tensorflow:No assets to write.


INFO:tensorflow:No assets to write.


INFO:tensorflow:SavedModel written to: model.h5/saved_model.pb


INFO:tensorflow:SavedModel written to: model.h5/saved_model.pb


In [16]:
loaded_model = tf.compat.v1.keras.experimental.load_from_saved_model("model.h5", custom_objects={'KerasLayer':hub.KerasLayer})

Instructions for updating:
The experimental save and load functions have been  deprecated. Please switch to `tf.keras.models.load_model`.


Instructions for updating:
The experimental save and load functions have been  deprecated. Please switch to `tf.keras.models.load_model`.


In [17]:
loaded_model.predict_classes(["The election in on Nov 3rd."])

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array([[0]], dtype=int32)